In [ ]:
!nvidia-smi

In [ ]:
# Стандартные библиотеки
import copy
import gc
import json
import logging
import os
import random
import shutil
import warnings

from IPython.display import Image
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt
import numpy as np
import tqdm
import torch
import torchvision

# Импорты из внешних пакетов (Detectron2)
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import load_coco_json, register_coco_instances
from detectron2.data import detection_utils as utils
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.modeling import build_model
from detectron2.utils.visualizer import Visualizer
import detectron2
import detectron2.data.transforms as T

In [ ]:
logger = logging.getLogger('detectron2')
logger.setLevel(logging.CRITICAL)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# Paths to Data

In [ ]:
annotations_train_path = 'data/train/annotations_train.json'
annotations_validation_path = 'data/validation/annotations_validation.json'
images_path = 'data/images'
binary_mask_path = 'data/binary_mask_train.npz'

In [ ]:
def clear_cache():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()


def check_gpu_availability():
    if torch.cuda.is_available():    
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())

        print('We will use the GPU:', torch.cuda.get_device_name(0))
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    return device


def show_images(file_id, dataset, metadata):
    example = dataset[file_id]
    image = utils.read_image(example["file_name"], format="RGB")
    plt.figure(figsize=(3,3),dpi=200)
    visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(example)
    plt.imshow(vis.get_image()[:, :,::-1])
    plt.show()


def f1_loss(y_actual, y_predicted):
    true_positive = np.sum(y_actual & y_predicted)
    false_positive = np.sum(~y_actual & y_predicted)
    false_negative = np.sum(y_actual & ~y_predicted)
    
    epsilon = 1e-7
    
    precision = true_positive / (true_positive + false_positive + epsilon)
    recall = true_positive / (true_positive + false_negative + epsilon)
    
    f1 = 2 * precision * recall / (precision + recall + epsilon)
    return f1


class custom_mapper:
    def __init__(self, cfg):
        self.transform_list = [
            T.ResizeShortestEdge(
                [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
                cfg.INPUT.MAX_SIZE_TEST),
            T.RandomBrightness(0.9, 1.1),
            T.RandomContrast(0.9, 1.1),
            T.RandomSaturation(0.9, 1.1),
            T.RandomLighting(0.9)
        ]
        print(f"[custom_mapper]: {self.transform_list}")

    def __call__(self, dataset_dict):
        dataset_dict = copy.deepcopy(dataset_dict)
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
    
        image, transforms = T.apply_transform_gens(self.transform_list, image)
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

        annos = [
            utils.transform_instance_annotations(obj, transforms, image.shape[:2])
            for obj in dataset_dict.pop("annotations")
            if obj.get("iscrowd", 0) == 0
        ]

        instances = utils.annotations_to_instances(annos, image.shape[:2])
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict


CHECKPOINTS_RESULTS = []

class F1Evaluator(DatasetEvaluator):
    def __init__(self):
        self.loaded_true = np.load(binary_mask_path)
        self.val_predictions = {}
        self.f1_scores = []
        
    def reset(self):
        self.val_predictions = {}
        self.f1_scores = []

    def process(self, inputs, outputs):
        for input, output in zip(inputs, outputs):
            filename = input["file_name"].split("/")[-1]
            if filename != "41_3.JPG":
                true = self.loaded_true[filename].reshape(-1)

                prediction = output['instances'].pred_masks.cpu().numpy()
                mask = np.add.reduce(prediction)
                mask = (mask > 0).reshape(-1)

                self.f1_scores.append(f1_loss(true, mask))

    def evaluate(self):
        global CHECKPOINTS_RESULTS
        result = np.mean(self.f1_scores)
        CHECKPOINTS_RESULTS.append(result)
        return {"meanF1": result}
    

class AugTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper(cfg))
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return F1Evaluator()

In [ ]:
DatasetCatalog.register("my_dataset_train",lambda: load_coco_json(annotations_train_path,
                                                                  image_root = images_path,
                                                                  dataset_name="my_dataset_train",
                                                                  extra_annotation_keys=['bbox_mode']))
DatasetCatalog.register("my_dataset_validation",lambda: load_coco_json(annotations_validation_path,
                                                                  image_root = images_path,
                                                                  dataset_name="my_dataset_validation",
                                                                  extra_annotation_keys=['bbox_mode']))

In [ ]:
dataset_train = DatasetCatalog.get("my_dataset_train")
dataset_validation = DatasetCatalog.get("my_dataset_validation")
print(f'Training dataset size (Images): {len(dataset_train)}')

metadata_train = MetadataCatalog.get("my_dataset_train")
metadata_validation = MetadataCatalog.get("my_dataset_validation")
print(f'Validation dataset size (Images): {len(dataset_validation)}')

In [ ]:
show_images(file_id=0, dataset=dataset_train, metadata=metadata_train)

# Model Configuration

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")) 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

In [ ]:
'''with open(annotations_train_path) as annotations_train:
    annotations = json.load(annotations_train)
    height, width = 10000, 10000
    for element in annotations["images"]:
        height = min(height, element["height"])
        width = min(width, element["width"])
    print(height, width)'''

In [ ]:
# Training and validation datasets
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

# Parallel data loading
cfg.DATALOADER.NUM_WORKERS = 4

# Image compression
cfg.INPUT.MIN_SIZE_TRAIN = 2160
cfg.INPUT.MAX_SIZE_TRAIN = 3130
cfg.INPUT.MIN_SIZE_TEST = cfg.INPUT.MIN_SIZE_TRAIN
cfg.INPUT.MAX_SIZE_TEST = cfg.INPUT.MAX_SIZE_TRAIN

# Decision threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1

# Validation frequency
cfg.TEST.EVAL_PERIOD = 1000

# Checkpoint frequency
cfg.SOLVER.CHECKPOINT_PERIOD = cfg.TEST.EVAL_PERIOD

# Blue Green Red (BGR) channel format
cfg.INPUT.FORMAT = 'BGR' 

# Batch size
cfg.SOLVER.IMS_PER_BATCH = 1

# Learning rate
cfg.SOLVER.BASE_LR = 0.01

# Learning rate reduction frequency
cfg.SOLVER.STEPS = (1500,)

# Learning rate reduction factor
cfg.SOLVER.GAMMA = 0.1

# Number of training iterations
cfg.SOLVER.MAX_ITER = 17000

# Number of classes in the dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Maximum number of words per page
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

# Model output path
cfg.OUTPUT_DIR = './output'

# Create the output directory if it doesn't exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Remove previous outputs
#%rm output/*

# Training the model

In [ ]:
trainer = AugTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
del trainer
clear_cache()

# Result check

In [ ]:
results = list(enumerate(CHECKPOINTS_RESULTS, start=1))
# файл с результатами валидации на каждом прогоне
with open("CHECKPOINTS_RESULTS.txt", "w") as f:
    f.write(str(results))

print(results)